<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M2_hatespeech_nlp_explainer_tm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trigger warning: This notebook contains words or language that are considered profane, vulgar, or offensive by some.

In [ ]:
# Install the tweet-preprocessor library (used for cleaning and preprocessing tweets)
!pip install tweet-preprocessor -q

# Install the latest version of gensim (a library for topic modeling and document similarity analysis)
!pip install -q -U gensim

# Install pyLDAvis (a Python library for interactive topic model visualization)
!pip install -q pyLDAvis

# Force reinstall numpy to version 1.22.4. This is currently necessary to ensure compatibility or fix certain issues.
!pip install --force-reinstall -q numpy==1.22.4

# Install LIME (Local Interpretable Model-Agnostic Explanations), a library for explaining machine learning model predictions
!pip install -q lime

# Install or update the imbalanced-learn library, useful for dealing with imbalanced datasets
!pip install -U imbalanced-learn -q


In [ ]:
import pandas as pd
import numpy as np
import preprocessor as prepro # twitter prepro
from tqdm.notebook import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
from lime.lime_text import LimeTextExplainer
from collections import OrderedDict

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

In [ ]:
# getting rid of annoying warnings from ipykernel
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [ ]:
# open file
data = pd.read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/twitter_hate.zip')

In [ ]:
#basic cleanup only for tweets
data['text_clean'] = data['tweet'].map(lambda t: prepro.clean(t))
data['text_clean'] = data['text_clean'].str.replace('#','')
data['text_clean'] = data['text_clean'].str.replace('rt','')

Spacy basics

In [ ]:
text = "this is a sentence about how much i would like to have a break in spanish they call it pausa"

In [ ]:
text_list = text.split(' ')

In [ ]:
' '.join(text_list)

In [ ]:
text_spacy = nlp(text)

In [ ]:
[(word.lemma_, word.pos_) for word in text_spacy if not word.is_stop]

In [ ]:
text_spacy.vector

In [ ]:
text_spacy = nlp('Aalborg Univ. is a nice place where Donaldo Trump has never been.')

In [ ]:
[(token, token.label_) for token in text_spacy.ents]

back to the project

In [ ]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline

clean_text = []

pbar = tqdm(total=len(data['text_clean']),position=0, leave=True)

for text in nlp.pipe(data['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text
         if token.is_alpha
         and not token.is_stop
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

In [ ]:
def preprocess_text(texts: pd.Series, nlp) -> list:
    """
    Preprocess a series of texts.

    Parameters:
    - texts: A pandas Series containing the text to be preprocessed.
    - nlp: A spaCy NLP model.

    Returns:
    - A list of preprocessed texts.

    Steps:
    - Clean twitter-specific characters using a predefined 'prepro' method.
    - Normalize the text by lowercasing and lemmatizing.
    - Remove punctuations, stopwords, and non-alphabet characters.
    """

    # Clean twitter-specific characters and other special characters
    texts_cleaned = texts.map(prepro.clean).str.replace('#', '')
    texts_cleaned = texts_cleaned.str.replace('#', '')

    # Initialize container for the cleaned texts
    clean_container = []

    # Use tqdm for a progress bar
    pbar = tqdm(total=len(texts_cleaned), position=0, leave=True)

    # Use spaCy's nlp.pipe for efficient text processing
    for doc in nlp.pipe(texts_cleaned, disable=["tagger", "parser", "ner"]):

        # Extract lemmatized tokens that are not punctuations, stopwords, or non-alphabetic
        tokens = [token.lemma_.lower() for token in doc
                  if token.is_alpha and not token.is_stop and not token.is_punct]

        clean_container.append(" ".join(tokens))

        pbar.update(1)

    return clean_container


In [ ]:
# apply all prepro-pipeline to texts
data['text_clean'] = text_prepro(data['tweet'])

In [ ]:
data

In [ ]:
# renaming and reordering

data_df = pd.DataFrame({'label':data['class'], 'text':data['text_clean']})

In [ ]:

data_df.label.value_counts().reset_index()

In [ ]:
# Count and reset index
data_chart = data_df.label.value_counts().reset_index().rename(columns={'label': 'Category', 'count': 'N Tweets'})

# Replace numerical categories with textual descriptions
data_chart['Category'] = data_chart['Category'].map({0: 'hate', 1: 'offensive', 2: 'nothing'})

# Plot the chart
chart = alt.Chart(data_chart).mark_bar(filled=True).encode(
    alt.X('N Tweets:Q', title='N Tweets'),
    alt.Y('Category:O', title='Category', sort='-x'),
    color=alt.Color('Category:N', legend=alt.Legend(title="Label Types"), scale=alt.Scale(
        domain=['hate', 'offensive', 'nothing'],
        range=['red', 'orange', 'green']
    ))
)

chart

In [ ]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data_df, data_df['label'])

In [ ]:
data_df_res['label'].value_counts()

In [ ]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(data_df_res['text'], y_res, test_size = 0.4, random_state = 42)

In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [ ]:
pipe.fit(X_train,y_train) # fit model

In [ ]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

In [ ]:
# run single prediction

t1 = ['you stupid fag bitch']

In [ ]:
# preprocess

t1_p = text_prepro(pd.Series(t1)) # note, we need to pack text up as pd.Series

In [ ]:
# predict

pipe.predict(t1_p)

In [ ]:
# Let's explain the result:

class_names = ["hate", "offensive", 'nothing']

explainer = LimeTextExplainer(class_names = class_names)

In [ ]:
exp = explainer.explain_instance(t1[0], pipe.predict_proba, num_features = 10, top_labels=3)

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
idx = 89

In [ ]:
exp = explainer.explain_instance(X_test.iloc[idx], pipe.predict_proba, num_features = 10, top_labels=3)

In [ ]:
print(y_test.iloc[idx])

In [ ]:
exp.show_in_notebook(text=True)

## Let's try a complex (black-box) model

In [ ]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
svd = TruncatedSVD(n_components = 100)
cls_xg = XGBClassifier()

pipe_xg = make_pipeline(tfidf, svd, cls_xg)

In [ ]:
pipe_xg.fit(X_train,y_train) # fit model

In [ ]:
# evaluate model performance on training set

y_eval = pipe_xg.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

In [ ]:
# evaluate model performance on test set

y_pred = pipe_xg.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
idx = 89

In [ ]:
exp = explainer.explain_instance(X_test.iloc[idx], pipe_xg.predict_proba, num_features = 10, top_labels=3)

In [ ]:
print(y_test.iloc[idx])

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
pred_xg = pipe_xg.predict(X_test)


In [ ]:
X_test_evaluation =pd.DataFrame(X_test)

In [ ]:
X_test_evaluation.index = range(len(X_test_evaluation))

In [ ]:
X_test_evaluation['y_test'] = list(y_test)

In [ ]:
X_test_evaluation['pred_xg'] = list(pred_xg)

In [ ]:
X_test_evaluation

In [ ]:
pd.crosstab(X_test_evaluation['y_test'],X_test_evaluation['pred_xg'], normalize='index')

## Topic model

In [ ]:
# preprocess texts (we need tokens)
tokens = []

for text in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in text
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV']
              and not token.is_stop
              and not token.is_punct]
  tokens.append(proj_tok)

In [ ]:
data['tokens'] = tokens

In [ ]:
data

We would like to know what things people are talking about when it is considerede "hatespeech"

In [ ]:
data_hate = data[data['class'] == 0]

In [ ]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data_hate['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data_hate['tokens']]

In [ ]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=10, workers = 4, passes=10)

In [ ]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

In [ ]:
 # Let's Visualize
pyLDAvis.display(lda_display)